In [1]:
import polars as pl

In [2]:
# Import incidents and format columns
df = pl.read_csv(
    "./data/incident_dump.csv",
).with_columns(
    pl.col("reported")
    .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%z")
    .dt.convert_time_zone("America/Chicago"),
    pl.col("reported_date").str.to_date("%Y-%m-%d"),
    pl.col("validated_location").str.split(",").cast(pl.List(pl.Float64)),
    pl.col("incident").str.to_lowercase(),
)

In [3]:
df.groupby(["incident"]).agg(pl.count()).sort("count", descending=True)

incident,count
str,u32
"""theft""",2961
"""information""",1370
"""found property…",916
"""lost property""",830
"""traffic violat…",534
"""information / …",493
"""liquor law vio…",441
"""medical call""",422
"""battery""",420


In [4]:
excluded_list = [
    "Fondling",
    "Medical Call",
    "Luring a Minor",
    "Lost Property",
    "Stalking",
    "Sexual Assault",
    "Dating",
    "Stalking",
    "Domestic",
    "Sex",
    "Found Property",
    "Mental Health",
    "Harassment by Electronic Means",
    "Well-Being",
    "Threatening Phone Call",
    "Medical Transport",
    "Warrant",
    "Lost Wallet",
    "Fire Alarm",
    "Chemical Spill",
    "Suspicious Mail",
    "Eavesdropping",
    "Sex Offense",
    "Sex Offender",
    "Sex Crime",
    "Domestic Aggravated Battery",
    "Dating Violence",
    "Harassing Messages",
]
df = df.filter(~pl.col("incident").str.contains("|".join(excluded_list)))
df.groupby(["incident"]).agg(pl.count()).sort("count", descending=True)

incident,count
str,u32
"""theft""",2961
"""information""",1370
"""found property…",916
"""lost property""",830
"""traffic violat…",534
"""information / …",493
"""liquor law vio…",441
"""medical call""",422
"""battery""",420


In [5]:
def list_to_parsed_set(unparsed_list: [str]):
    unparsed_list.sort()
    parsed_set = set()
    for element in unparsed_list:
        if "/" in element:
            for p in element.split(r"\s?/\s?"):
                fmt_element = p.strip().lower()
                if p:
                    parsed_set.add(fmt_element)
        else:
            fmt_element = element.strip().lower()
            parsed_set.add(fmt_element)
    return parsed_set

In [6]:
print(list_to_parsed_set(df["incident"].to_list()))

{'battery / criminal trespass to land', 'information / unlawful use of weapon', 'motor vehicle recovery', 'assist other agency / aggravated battery', 'information / well-being check', 'armed robbery arrest', 'traffic accident / hit and run', 'aggravated battery of a police officer', 'information / unlawful discharge of a weapon', 'non-criminal incident', 'traffic violation / warrant', 'missing person', 'illegal consumption by minor', 'information / fire', 'robbery / strong arm', 'criminal damage to property / criminal defacement', 'recovered motor vehicle', 'hate crime / telephone threat', 'information / domestic issue', 'information / trespass to vehicle', 'criminal trespass to motor vehicle', 'recovered vehicle plate', 'assist other agency / mental health transport', 'information / criminal damage to property', 'theft from motor vehicle / public way', 'possession of drug paraphernalia', 'information / traffic crash / damage', 'information / shot fired', 'deliver narcotic', 'informati